# RAG using csv-retriever 

https://python.langchain.com/docs/integrations/toolkits/csv
https://www.youtube.com/watch?v=xQ3mZhw69bc&t=35s
https://colab.research.google.com/drive/1BEZ_qgtVqSmOmCTuhHs7lHiYB5M5_myg?usp=sharing#scrollTo=wFjXiYDjUzZ8

In [32]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI

## Zero Shot Agent

In [51]:
agent = create_csv_agent(
    OpenAI(temperature=0),
    "faker/dataset_3.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [52]:
agent

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'df_head': "|    | Visit ID   | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Visit Date   | Patient ID   | Diagnosis Category                    | Diagnosis Sub Category   | 

In [5]:
agent.run("How many man are in the dataset?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe where the patient's gender is male.
Action: python_repl_ast
Action Input: len(df[df['Patient Gender'] == 'Male'])47104710 is the number of male patients in the dataset.
Final Answer: There are 4710 male patients in the dataset.

> Finished chain.


'There are 4710 male patients in the dataset.'

In [7]:
agent.run("How many patients have cancer?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the Diagnosis Category is "Cancer" and then count the number of unique Patient IDs.
Action: python_repl_ast
Action Input: df[df["Diagnosis Category"] == "Cancer"]["Patient ID"].nunique()710710 patients have cancer.
Final Answer: 710

> Finished chain.


'710'

In [9]:
agent.run("How many women with breast cancer?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the Diagnosis Category is "Cancer" and the Patient Gender is "Female".
Action: python_repl_ast
Action Input: df[(df["Diagnosis Category"] == "Cancer") & (df["Patient Gender"] == "Female")]        Visit ID                                        Description  \
4      VAF235393  The patient presented with a diagnosis of brea...   
19     VKH924729  The patient presented with a diagnosis of brea...   
27     VTD070256  Today I saw a patient with suspected breast ca...   
49     VVH026262  The patient with suspected ovarian cancer unde...   
73     VDW581539  The patient has been diagnosed with breast can...   
...          ...                                                ...   
11451  VLK193060  The patient presented with a diagnosis of lung...   
11481  VCI669065  Today, I reviewed the case of a patient with s...   
11557  VXA645061  I reviewed the patient's history and ordered a..

'There are 576 women with breast cancer in the dataframe.'

In [45]:
agent.run("Provide the data for women with breast cancer that allow their data to be used for research purposes.")



> Entering new AgentExecutor chain...
Thought: We need to filter the dataframe to only include rows where the Diagnosis Category is "Cancer" and the Patient Gender is "Female".
Action: python_repl_ast
Action Input: df[(df["Diagnosis Category"] == "Cancer") & (df["Patient Gender"] == "Female")]        Visit ID                                        Description  \
4      VAF235393  The patient presented with a diagnosis of brea...   
19     VKH924729  The patient presented with a diagnosis of brea...   
27     VTD070256  Today I saw a patient with suspected breast ca...   
49     VVH026262  The patient with suspected ovarian cancer unde...   
73     VDW581539  The patient has been diagnosed with breast can...   
...          ...                                                ...   
11451  VLK193060  The patient presented with a diagnosis of lung...   
11481  VCI669065  Today, I reviewed the case of a patient with s...   
11557  VXA645061  I reviewed the patient's history and ordered a.

'df[(df["Diagnosis Category"] == "Cancer") & (df["Patient Gender"] == "Female")]'

## OpenAI Functions

In [11]:
agent = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    "faker/dataset_3.csv",
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [24]:
agent

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessage(content="\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nThis is the result of `print(df.head())`:\n|    | Visit ID   | Description                                                                                                                                                                                                                               

In [14]:
agent.run("Provide the data for women with breast cancer.")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Diagnosis Category'] == 'Cancer'][df['Patient Gender'] == 'Female']"}`


        Visit ID                                        Description  \
4      VAF235393  The patient presented with a diagnosis of brea...   
19     VKH924729  The patient presented with a diagnosis of brea...   
27     VTD070256  Today I saw a patient with suspected breast ca...   
49     VVH026262  The patient with suspected ovarian cancer unde...   
73     VDW581539  The patient has been diagnosed with breast can...   
...          ...                                                ...   
11451  VLK193060  The patient presented with a diagnosis of lung...   
11481  VCI669065  Today, I reviewed the case of a patient with s...   
11557  VXA645061  I reviewed the patient's history and ordered a...   
11579  VOV310319  Today, I reviewed the case of a patient who pr...   
11654  VVU252443  The patient presented with symptoms

<string>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Here is the data for women with breast cancer:

| Visit ID   | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Visit Date   | Patient ID   | Diagnosis Category | Diagnosis Sub Category | Treatment Category     | Treatment Sub Category | New Patient | Consulting Physician | Patient Name          | Patient Gender | Patient Age | Patient Age Range | Patient Blood Type | Patient Insurance Number | Patient Phone  | Patient Address    

"Here is the data for women with breast cancer:\n\n| Visit ID   | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Visit Date   | Patient ID   | Diagnosis Category | Diagnosis Sub Category | Treatment Category     | Treatment Sub Category | New Patient | Consulting Physician | Patient Name          | Patient Gender | Patient Age | Patient Age Range | Patient Blood Type | Patient Insurance Number | Patient Phone  | Patient Address 

In [21]:
agent.run("How many women with breast cancer are in the dataset?.")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['Diagnosis Category'] == 'Cancer') & (df['Patient Gender'] == 'Female')].shape[0]"}`


576There are 576 women with breast cancer in the dataset.

> Finished chain.


'There are 576 women with breast cancer in the dataset.'

In [20]:
agent.run("How many unique women with breast cancer are in the dataset?.")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Diagnosis Category'] == 'Cancer'][df['Patient Gender'] == 'Female']['Patient ID'].nunique()"}`


410

<string>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


There are 410 unique women with breast cancer in the dataset.

> Finished chain.


'There are 410 unique women with breast cancer in the dataset.'

In [17]:
agent.run("How many women with breast cancer allow their data to be used for research purposes.")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['Diagnosis Category'] == 'Cancer') & (df['Patient Gender'] == 'Female') & (df['Intended Purposes'].str.contains('Research'))]['Patient ID'].nunique()"}`


330There are 330 women with breast cancer who allow their data to be used for research purposes.

> Finished chain.


'There are 330 women with breast cancer who allow their data to be used for research purposes.'

In [18]:
agent.run("How many women with breast cancer allow their data to be used for care purposes.")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['Diagnosis Category'] == 'Cancer') & (df['Patient Gender'] == 'Female') & (df['Intended Purposes'].str.contains('Care'))]['Patient ID'].nunique()"}`


410There are 410 women with breast cancer who allow their data to be used for care purposes.

> Finished chain.


'There are 410 women with breast cancer who allow their data to be used for care purposes.'

In [23]:
agent.run("Provide the medical data for patient PPD253419. Provide the total number of records plus description and date for each visit.")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Patient ID'] == 'PPD253419'][['Visit ID', 'Description', 'Visit Date']]"}`


       Visit ID                                        Description  Visit Date
2     VKT437745  The patient presented with delayed speech deve...  2017-01-01
4338  VAB432603  Today, I saw a young patient who was referred ...  2017-10-01The medical data for patient PPD253419 includes two visits. Here are the details for each visit:

1. Visit ID: VKT437745
   Description: The patient presented with delayed speech development, exhibiting difficulties in verbal communication appropriate for their age. Following evaluation, it was determined that the most suitable course of action would be to initiate outpatient speech therapy as part of their rehabilitation plan. This intervention aims to improve the patient's speech and language skills through specialized exercises and techniques tailored to their individual needs. Regula

"The medical data for patient PPD253419 includes two visits. Here are the details for each visit:\n\n1. Visit ID: VKT437745\n   Description: The patient presented with delayed speech development, exhibiting difficulties in verbal communication appropriate for their age. Following evaluation, it was determined that the most suitable course of action would be to initiate outpatient speech therapy as part of their rehabilitation plan. This intervention aims to improve the patient's speech and language skills through specialized exercises and techniques tailored to their individual needs. Regular sessions will be scheduled to monitor progress and make necessary adjustments to facilitate optimal improvement in the patient's communication abilities.\n   Visit Date: 2017-01-01\n\n2. Visit ID: VAB432603\n   Description: Today, I saw a young patient who was referred to me for evaluation and management of their delayed speech development. After a thorough assessment, it was determined that the p

In [25]:
agent.run("Summarize the medical history of patient PAP587444.")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Patient ID'] == 'PAP587444']"}`


        Visit ID                                        Description  \
466    VTX396320  I reviewed the patient's history of chronic ba...   
4333   VOQ276808  The patient presented with symptoms indicative...   
8215   VNC798793  The patient presented with symptoms of severe ...   
9016   VZZ736903  The patient presented to the emergency room wi...   
9511   VTP523682  The patient presented with severe hypertension...   
10370  VJY034996  The patient presented with symptoms of persist...   

       Visit Date Patient ID           Diagnosis Category  \
466    2017-01-30  PAP587444  Orthopedic/ Musculoskeletal   
4333   2017-10-01  PAP587444                       Cancer   
8215   2018-06-04  PAP587444  Orthopedic/ Musculoskeletal   
9016   2018-07-23  PAP587444           Respiratory System   
9511   2018-08-22  PAP587444               Morbid Obesity   
10370  20

"The medical history of patient PAP587444 includes the following visits:\n\n1. Visit ID: VTX396320\n   - Description: The patient's history of chronic back pain was reviewed. Diagnostic imaging (MRI) was performed.\n   - Diagnosis Category: Orthopedic/ Musculoskeletal\n   - Diagnosis Sub Category: Back Pain\n   - Treatment Category: Diagnostic Imaging, Screening and Testing\n\n2. Visit ID: VOQ276808\n   - Description: The patient presented with symptoms indicative of breast cancer. Emergency room services were provided.\n   - Diagnosis Category: Cancer\n   - Diagnosis Sub Category: Breast Cancer\n   - Treatment Category: Acute Medical Services - Outpatient\n\n3. Visit ID: VNC798793\n   - Description: The patient presented with symptoms of severe TMJ (Temporal Mandibular Joint) disorder. Botox injection was administered.\n   - Diagnosis Category: Orthopedic/ Musculoskeletal\n   - Diagnosis Sub Category: TMJ - Temporal Mandibular Joint\n   - Treatment Category: Pharmacy/Prescription Drug

In [28]:
agent.run("How many visits cam on February 2, 2017?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Visit Date'] == '2017-02-02'].shape[0]"}`


15There were 15 visits on February 2, 2017.

> Finished chain.


'There were 15 visits on February 2, 2017.'

In [29]:
agent.run("How many visits cam on February 30, 2017?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Visit Date'] == '2017-02-30'].shape[0]"}`


0There were no visits on February 30, 2017.

> Finished chain.


'There were no visits on February 30, 2017.'

In [31]:
agent

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessage(content="\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nThis is the result of `print(df.head())`:\n|    | Visit ID   | Description                                                                                                                                                                                                                               

## Custom Template

In [36]:
custom_prompt_prefix = 'You are a helpful agent that specialises in information retrieval and question answering. First, retrieve relevant information. Second, answer the question or execute the demanded task. '

agent = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    "faker/dataset_3.csv",
    verbose=True,
    prefix=custom_prompt_prefix,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [35]:
from IPython.display import Markdown, display

# Display the Markdown formatted text
display(Markdown(agent.run("Summarize the medical history of patient PAP587444.")))



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Patient ID'] == 'PAP587444']"}`
responded: To summarize the medical history of patient PAP587444, we need to retrieve relevant information from the dataset.

        Visit ID                                        Description  \
466    VTX396320  I reviewed the patient's history of chronic ba...   
4333   VOQ276808  The patient presented with symptoms indicative...   
8215   VNC798793  The patient presented with symptoms of severe ...   
9016   VZZ736903  The patient presented to the emergency room wi...   
9511   VTP523682  The patient presented with severe hypertension...   
10370  VJY034996  The patient presented with symptoms of persist...   

       Visit Date Patient ID           Diagnosis Category  \
466    2017-01-30  PAP587444  Orthopedic/ Musculoskeletal   
4333   2017-10-01  PAP587444                       Cancer   
8215   2018-06-04  PAP587444  Orthopedic/ Musculoskeletal   
9016   

Based on the medical history of patient PAP587444, here is a summary:

1. Visit ID: VTX396320
   - Description: The patient has a history of chronic back pain. Diagnostic imaging, specifically an MRI, was performed to further evaluate the condition.
   - Diagnosis Category: Orthopedic/ Musculoskeletal
   - Diagnosis Sub Category: Back Pain
   - Treatment Category: Diagnostic Imaging, Screening and Testing
   - Treatment Sub Category: MRI

2. Visit ID: VOQ276808
   - Description: The patient presented with symptoms indicative of breast cancer. The diagnosis was confirmed, and the patient received acute medical services as an outpatient. The patient also visited the emergency room.
   - Diagnosis Category: Cancer
   - Diagnosis Sub Category: Breast Cancer
   - Treatment Category: Acute Medical Services - Outpatient, Emergency Room

3. Visit ID: VNC798793
   - Description: The patient presented with severe symptoms related to the TMJ (Temporal Mandibular Joint). Botox injection was administered as part of the treatment.
   - Diagnosis Category: Orthopedic/ Musculoskeletal
   - Diagnosis Sub Category: TMJ - Temporal Mandibular Joint
   - Treatment Category: Pharmacy/Prescription Drugs
   - Treatment Sub Category: Botox Injection

4. Visit ID: VZZ736903
   - Description: The patient visited the emergency room with respiratory system-related symptoms.
   - Diagnosis Category: Respiratory System
   - Treatment Category: Acute Medical Services - Outpatient, Emergency Room

5. Visit ID: VTP523682
   - Description: The patient presented with severe hypertension. The treatment involved general surgery.
   - Diagnosis Category: Morbid Obesity
   - Diagnosis Sub Category: Hypertension
   - Treatment Category: General Surgery

6. Visit ID: VJY034996
   - Description: The patient presented with persistent symptoms related to lung cancer. Surgery was performed as part of the cancer treatment.
   - Diagnosis Category: Cancer
   - Diagnosis Sub Category: Lung Cancer
   - Treatment Category: Cancer Treatment
   - Treatment Sub Category: Surgery

Please note that this is a summary of the available information in the dataset for patient PAP587444.

In [47]:
custom_prompt_prefix = 'You are a helpful agent that specializes in information retrieval and question answering. First, retrieve relevant information. Second, answer the question or execute the demanded task. This includes interpreting the retrieved information. Provide a text output alongside the data.'

agent = create_csv_agent(
    ChatOpenAI(temperature=0.3),
    "faker/dataset_3.csv",
    verbose=True,
    prefix=custom_prompt_prefix,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [48]:
from IPython.display import Markdown, display

# Display the Markdown formatted text
display(Markdown(agent.run("Summarize the medical history of patient PAP587444.")))



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Patient ID']=='PAP587444']"}`


        Visit ID                                        Description  \
466    VTX396320  I reviewed the patient's history of chronic ba...   
4333   VOQ276808  The patient presented with symptoms indicative...   
8215   VNC798793  The patient presented with symptoms of severe ...   
9016   VZZ736903  The patient presented to the emergency room wi...   
9511   VTP523682  The patient presented with severe hypertension...   
10370  VJY034996  The patient presented with symptoms of persist...   

       Visit Date Patient ID           Diagnosis Category  \
466    2017-01-30  PAP587444  Orthopedic/ Musculoskeletal   
4333   2017-10-01  PAP587444                       Cancer   
8215   2018-06-04  PAP587444  Orthopedic/ Musculoskeletal   
9016   2018-07-23  PAP587444           Respiratory System   
9511   2018-08-22  PAP587444               Morbid Obesity   
10370  2018

Patient PAP587444 has a medical history that includes the following visits:

1. **Visit 1 (2017-01-30):**
   - Diagnosis Category: Orthopedic/ Musculoskeletal
   - Diagnosis Sub Category: Back Pain
   - Treatment Category: Diagnostic Imaging, Screening and Testing (MRI)
   - Consulting Physician: Dr. Shelly Hunt

2. **Visit 2 (2017-10-01):**
   - Diagnosis Category: Cancer
   - Diagnosis Sub Category: Breast Cancer
   - Treatment Category: Acute Medical Services - Outpatient (Emergency Room)
   - Consulting Physician: Dr. James Barber

3. **Visit 3 (2018-06-04):**
   - Diagnosis Category: Orthopedic/ Musculoskeletal
   - Diagnosis Sub Category: TMJ - Temporal Mandibular Joint
   - Treatment Category: Pharmacy/Prescription Drugs (Botox Injection)
   - Consulting Physician: Dr. Jerry Daniels

4. **Visit 4 (2018-07-23):**
   - Diagnosis Category: Respiratory System
   - Treatment Category: Acute Medical Services - Outpatient (Emergency Room)
   - Consulting Physician: Dr. Eddie Young

5. **Visit 5 (2018-08-22):**
   - Diagnosis Category: Morbid Obesity
   - Diagnosis Sub Category: Hypertension
   - Treatment Category: General Surgery
   - Consulting Physician: Dr. James Barber

6. **Visit 6 (2018-10-11):**
   - Diagnosis Category: Cancer
   - Diagnosis Sub Category: Lung Cancer
   - Treatment Category: Cancer Treatment (Surgery)
   - Consulting Physician: Dr. Shelly Hunt

The patient, Carolyn Johnson, is a 37-year-old female with blood type A-. She has the same insurance number, phone number, address, occupation (Advertising copywriter), and emergency contact (Teresa Rogers) across all visits. The intended purposes of the visits include care, research, insurance, support, and other related purposes.

In [50]:
import pandas as pd 

df = pd.read_csv('faker/dataset_3.csv', index_col='Visit ID')
df[df['Patient ID']=='PAP587444']

,Description,Visit Date,Patient ID,Diagnosis Category,Diagnosis Sub Category,Treatment Category,Treatment Sub Category,New Patient,Consulting Physician,Patient Name,Patient Gender,Patient Age,Patient Age Range,Patient Blood Type,Patient Insurance Number,Patient Phone,Patient Address,Patient Occupation,Patient Emergency Contact,Intended Purposes
Visit ID,,,,,,,,,,,,,,,,,,,,
VTX396320,I reviewed the patient's history of chronic ba...,2017-01-30,PAP587444,Orthopedic/ Musculoskeletal,Back Pain,"Diagnostic Imaging, Screening and Testing",MRI,True,Dr. Shelly Hunt,Carolyn Johnson,Female,36,31-40,A-,R630520434,17943741823,Henni-Hermighausen-Platz 6\n15101 Bad Mergentheim,Advertising copywriter,"Teresa Rogers, 017846726524.","['Care', 'Research', 'Insurance', 'Support', '..."
VOQ276808,The patient presented with symptoms indicative...,2017-10-01,PAP587444,Cancer,Breast Cancer,Acute Medical Services - Outpatient,Emergency Room,False,Dr. James Barber,Carolyn Johnson,Female,36,31-40,A-,R630520434,17943741823,Henni-Hermighausen-Platz 6\n15101 Bad Mergentheim,Advertising copywriter,"Teresa Rogers, 017846726524.","['Care', 'Research', 'Insurance', 'Support', '..."
VNC798793,The patient presented with symptoms of severe ...,2018-06-04,PAP587444,Orthopedic/ Musculoskeletal,TMJ - Temporal Mandibular Joint,Pharmacy/Prescription Drugs,Botox Injection,False,Dr. Jerry Daniels,Carolyn Johnson,Female,37,31-40,A-,R630520434,17943741823,Henni-Hermighausen-Platz 6\n15101 Bad Mergentheim,Advertising copywriter,"Teresa Rogers, 017846726524.","['Care', 'Research', 'Insurance', 'Support', '..."
VZZ736903,The patient presented to the emergency room wi...,2018-07-23,PAP587444,Respiratory System,NaN,Acute Medical Services - Outpatient,Emergency Room,False,Dr. Eddie Young,Carolyn Johnson,Female,37,31-40,A-,R630520434,17943741823,Henni-Hermighausen-Platz 6\n15101 Bad Mergentheim,Advertising copywriter,"Teresa Rogers, 017846726524.","['Care', 'Research', 'Insurance', 'Support', '..."
VTP523682,The patient presented with severe hypertension...,2018-08-22,PAP587444,Morbid Obesity,Hypertension,General Surgery,NaN,False,Dr. James Barber,Carolyn Johnson,Female,37,31-40,A-,R630520434,17943741823,Henni-Hermighausen-Platz 6\n15101 Bad Mergentheim,Advertising copywriter,"Teresa Rogers, 017846726524.","['Care', 'Research', 'Insurance', 'Support', '..."
VJY034996,The patient presented with symptoms of persist...,2018-10-11,PAP587444,Cancer,Lung Cancer,Cancer Treatment,Surgery,False,Dr. Shelly Hunt,Carolyn Johnson,Female,37,31-40,A-,R630520434,17943741823,Henni-Hermighausen-Platz 6\n15101 Bad Mergentheim,Advertising copywriter,"Teresa Rogers, 017846726524.","['Care', 'Research', 'Insurance', 'Support', '..."
